# SQL Querying

This notebook can be used to query tables in the SBA database. In order to use this notebook, you will need to set an environment variable 'SBA_DWH' to the database connection string. If you do not have the credentials, please slack us at #datasci-sba channel.

**It is best practice to not hard code database URI strings directly in notebooks or code as when we push to Github, that would mean credentials are public for anyone to see.**

In [1]:
import os

import pandas as pd
pd.options.display.max_columns = 999
import sqlalchemy as sqla
from sqlalchemy import create_engine

SBA_DWH = os.getenv('SBA_DWH')
engine = create_engine(SBA_DWH)

Below are the tables that currently exist in the database:

In [2]:
QUERY = """
select *
from information_schema.tables
where table_schema not in ('information_schema', 'pg_catalog', 'public')
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(100)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,postgres,data_ingest,sba__foia_7a_2010_present,BASE TABLE,None,None,None,None,None,YES,NO,None
1,postgres,data_ingest,census__zip_business_patterns,BASE TABLE,None,None,None,None,None,YES,NO,None
2,postgres,data_ingest,sba__foia_7a_1991_1999,BASE TABLE,None,None,None,None,None,YES,NO,None
3,postgres,data_ingest,sba__foia_7a_2000_2009,BASE TABLE,None,None,None,None,None,YES,NO,None
4,postgres,stg_analytics,sba_sfdo,BASE TABLE,None,None,None,None,None,YES,NO,None
5,postgres,data_ingest,sba__foia_504_1991_present,BASE TABLE,None,None,None,None,None,YES,NO,None
6,postgres,data_ingest,irs__zip_data,BASE TABLE,None,None,None,None,None,YES,NO,None
7,postgres,stg_analytics,irs_income,BASE TABLE,None,None,None,None,None,YES,NO,None
8,postgres,stg_analytics,census_naics,BASE TABLE,None,None,None,None,None,YES,NO,None
9,postgres,trg_analytics,sba_zip_level,BASE TABLE,None,None,None,None,None,YES,NO,None


### List all the CDC and Bank names in the data

In [3]:
QUERY = """
SELECT cdc_name, cdc_city, cdc_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE cdc_name IS NOT NULL
    GROUP BY cdc_name, cdc_city, cdc_state ORDER BY count
LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(100)

,cdc_name,cdc_city,cdc_state,count
0,MISSNG/INACTV LENDER,None,None,1
1,Coastal Business Finance,Santa Maria,CA,1
2,Northwest Business Development Association,Spokane Valley,WA,2
3,Cen Cal Business Finance Group,Fresno,CA,4
4,Success Capital Expansion & Development Corpor...,Modesto,CA,5
5,"AMPAC Tri-State CDC, Inc.",Grand Terrace,CA,6
6,SBA Denver Finance Center,Denver,CO,9
7,Business Finance Capital,Los Angeles,CA,9
8,MISSNG/INACTV LENDER,WASHINGTON,DC,10
9,MISSNG/INACTV LENDER,Salinas,CA,10


In [4]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name IS NOT NULL 
    GROUP BY bank_name, bank_city, bank_state ORDER BY count DESC
    LIMIT 250
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(100)

,bank_name,bank_city,bank_state,count
0,"Wells Fargo Bank, National Association",SIOUX FALLS,SD,8419
1,"Bank of America, National Association",CHARLOTTE,NC,3276
2,U.S. Bank National Association,Cincinnati,OH,2208
3,Bank of Hope,LOS ANGELES,CA,1903
4,Umpqua Bank,ROSEBURG,OR,1360
5,"JPMorgan Chase Bank, National Association",COLUMBUS,OH,1265
6,Comerica Bank,DALLAS,TX,1031
7,Heritage Bank of Commerce,SAN JOSE,CA,1001
8,"Citibank, N.A.",SIOUX FALLS,SD,992
9,"Zions Bank, A Division of",SALT LAKE CITY,UT,901


### Run various keyword queries that hit all the names in CACFDIs.xlsx

This query results in a possible match for `Valley Economic Development Center, Van Nuys`

In [5]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%valley%%' 
    GROUP BY bank_name, bank_city, bank_state ORDER by bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Central Valley Community Bank,FRESNO,CA,9
1,Mission Valley Bank,SUN VALLEY,CA,2
2,Oak Valley Community Bank,OAKDALE,CA,4
3,Pacific Valley Bank,SALINAS,CA,1
4,Premier Valley Bank,FRESNO,CA,8
5,Scott Valley Bank,YREKA,CA,2
6,Silicon Valley Bank,SANTA CLARA,CA,16
7,Tri-Valley Bank,SAN RAMON,CA,17
8,Valley Community Bank,PLEASANTON,CA,3
9,Valley Economic Development Corp.,Van Nuys,CA,26


In [6]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%core%%'
    GROUP by bank_name, bank_city, bank_state ORDER by count DESC
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [7]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%accion%%'
    GROUP by bank_name, bank_city, bank_state ORDER by count DESC
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [8]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%america%%'
    GROUP BY bank_name, bank_city, bank_state ORDER by bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,American Continental Bank,CITY OF INDUSTRY,CA,5
1,American Express Centurion Bank,SALT LAKE CITY,UT,11
2,American National Bank,OMAHA,NE,4
3,American River Bank,SACRAMENTO,CA,29
4,American Riviera Bank,SANTA BARBARA,CA,2
5,AmericanWest Bank,ONTARIO,CA,1
6,Americas United Bank,GLENDALE,CA,1
7,Banco Popular North America,NEW YORK CITY,NY,85
8,"Bank of America California, National Association",SAN FRANCISCO,CA,767
9,"Bank of America, National Association",CHARLOTTE,NC,3276


This query finds two possible matches. `TMC Development Working Solutions, San Francisco` and `Valley Economic Development Center, Van Nuys`. 
The latter matches other keyword searches. Google cross-referencing on TMC and The Mortgage Capital Devlopment is not conclusive.

In [9]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%development%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Community Development Transportation Lending S...,Washington DC,DC,2
1,Empire State Certified Development Corporation,Albany,NY,1
2,The Mortgage Capital Development Corporation,San Francisco,CA,9
3,Valley Economic Development Corp.,Van Nuys,CA,26


In [10]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%village%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


This yields a possible match for `Bankers' Small Business CDC of San Diego`. Google hits for the two names are very similar.

In [11]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%business%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,"Business Lenders, LLC",HARTFORD,CT,23
1,"Business Loan Center, LLC",GREENVILLE,SC,59
2,CDC Small Business Finance Corporation,San Diego,CA,29
3,CIT Small Business Lending Corporation,LIVINGSTON,NJ,1
4,"CitiCapital Small Business Finance, Inc.",IRVING,TX,131
5,Citizens Business Bank,ONTARIO,CA,1
6,Commonwealth Business Bank,LOS ANGELES,CA,4
7,"CRF Small Business Loan Company, LLC",MINNEAPOLIS,MN,22
8,"Crossroads Small Business Solutions, LLC",Rancho Cucamonga,CA,348
9,"Freedom Small Business Lending BIDCO, Inc.",NEWARK,DE,1


In [12]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%union%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,"MUFG Union Bank, National Association",SAN FRANCISCO,CA,554
1,National Credit Union Administration,ALEXANDRIA,VA,6


This yields an exact match for `Beneficial State Bank, Oakland`.

In [13]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%beneficial%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Beneficial Bank,PHILADELPHIA,PA,3
1,Beneficial State Bank,OAKLAND,CA,45


This yields an exact match for `Community Bank of the Bay, Oakland` and a very close match for `Santa Cruz Community Credit Union, Santa Cruz`

In [14]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%community%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,BAC Community Bank,STOCKTON,CA,197
1,Central Valley Community Bank,FRESNO,CA,9
2,Community 1st Bank,AUBURN,CA,4
3,Community Bank,PASADENA,CA,8
4,Community Bank of the Bay,OAKLAND,CA,102
5,"Community Banks of Colorado, A Division of",GREENWOOD VILLAGE,CO,36
6,Community Development Transportation Lending S...,Washington DC,DC,2
7,"Community West Bank, National Association",GOLETA,CA,120
8,First Community Bank,SANTA ROSA,CA,184
9,First U.S. Community CU,SACRAMENTO,CA,2


In [15]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%farm%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Farmers & Merchants Bank & Trust,BURLINGTON,IA,1
1,Farmers & Merchants Bank of Central California,LODI,CA,4


This one matches picks up `Valley Economic Development Center, Van Nuys`, which a Google search seems to indicate is a match for `Valley Economic Development Corp.`

In [16]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%economic%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Valley Economic Development Corp.,Van Nuys,CA,26


In [17]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%home%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,First Home Bank,SEMINOLE,FL,43
1,"Home Loan Investment Bank, F.S.B.",WARWICK,RI,1
2,HomeStreet Bank,SEATTLE,WA,1


In [18]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%housing%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [19]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%financ%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name 
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,CDC Small Business Finance Corporation,San Diego,CA,29
1,Choice Financial Group,FARGO,ND,2
2,"CitiCapital Small Business Finance, Inc.",IRVING,TX,131
3,First Bank Financial Centre,OCONOMOWOC,WI,3
4,First Financial Bank,CINCINNATI,OH,1
5,First Financial Bank,EL DORADO,AR,2
6,"First Financial Bank, National Association",HAMILTON,OH,1
7,GE Capital Small Business Finance Corporation,HOUSTON,TX,393
8,"Harvest Small Business Finance, LLC",Laguna Hills,CA,21
9,"Newtek Small Business Finance, Inc.",LAKE SUCCESS,NY,60


In [20]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%choice%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Choice Financial Group,FARGO,ND,2


In [21]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%income%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [22]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%neighborhood%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [23]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%one%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,"Capital One Bank (USA), National Association",GLEN ALLEN,VA,22
1,"Capital One, National Association",MCLEAN,VA,842
2,"CenterStone SBA Lending, Inc.",LOS ANGELES,CA,1
3,"Credit One Bank, National Association",LAS VEGAS,NV,4
4,MutualOne Bank,FRAMINGHAM,MA,1
5,Vision One CU,SACRAMENTO,CA,38


In [24]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%opening%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [25]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%opportunity%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [26]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%trail%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


In [27]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%poverty%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count


The following yields an exact match for `SCE FCU, Irwindale`.

In [28]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%sce%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,SCE FCU,IRWINDALE,CA,1


In [29]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%alliance%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Western Alliance Bank,PHOENIX,AZ,277


In [30]:
QUERY = """
SELECT bank_name, bank_city, bank_state, count(*)
    FROM stg_analytics.sba_sfdo WHERE bank_name ILIKE '%%federal%%'
    GROUP BY bank_name, bank_city, bank_state ORDER BY bank_name
    LIMIT 25
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(25)

,bank_name,bank_city,bank_state,count
0,Federal Deposit Insurance Corporation,WASHINGTON,DC,627
